In [2]:
import os
from groq import Groq

In [3]:
groq_api_key = "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"
client = Groq(api_key=groq_api_key)

In [32]:
def summarize_text(text, max_words, retries=3):
    prompt = f"Summarize the following text in approximately {max_words} words. It's crucial to keep the summary as close to {max_words} words as possible:\n\n{text}\n\nSummary:"

    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": f"You are a helpful assistant that provides concise summaries. Always aim to keep your summaries as close to the requested word count as possible."},
            {"role": "user", "content": prompt}
        ],
        model="mixtral-8x7b-32768",
        temperature=0.7,
        top_p=1,
        max_tokens=max_words * 2
    )

    return chat_completion.choices[0].message.content

In [5]:
import PyPDF2
import re

In [6]:
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

In [33]:
def extract_pdf_content(base_path, pdf_file):
    full_path = os.path.join(base_path, pdf_file)
    with open(full_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text() + '\n'
    return clean_text(text)

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

In [34]:
def splitter_chunks(text, filename):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    langchain_doc = Document(page_content=text, metadata={"source": filename})
    return text_splitter.split_documents([langchain_doc])

In [35]:
base_path = 'F:/Super AI SS4/AI-builder/Large-Language-Models-(LLMs)/try-it/Text-Summarization/Dataset/'
pdf_files = ['document1.pdf', 'document2.pdf', 'document3.pdf']

all_chunks = []

for pdf_file in pdf_files:
    print(f"Processing {pdf_file}...")
    content = extract_pdf_content(base_path, pdf_file)
    chunks = splitter_chunks(content, pdf_file)
    all_chunks.extend(chunks)

print(f"Total number of chunks across all documents: {len(all_chunks)}")

Processing document1.pdf...
Processing document2.pdf...
Processing document3.pdf...
Total number of chunks across all documents: 18


In [ ]:
'''
documents = []

for i, pdf_file in enumerate(pdf_files, 1):
    content = extract_pdf_content(base_path, pdf_file)
    document = {
        'id': i,
        'filename': pdf_file,
        'content': content,
        'length': len(content)
    }
    documents.append(document)

for doc in documents:
    print(f"Document ID: {doc['id']}")
    print(f"Filename: {doc['filename']}")
    print(f"Content length: {doc['length']} characters")
    print(f"Preview: {doc['content'][:100]}...")
    print("\n" + "="*50 + "\n")
'''

In [11]:
import time

In [36]:
chunk_summaries = []

for i, chunk in enumerate(all_chunks, 1):
    print(f"Summarizing chunk {i}/{len(all_chunks)}...")
    summary = summarize_text(chunk.page_content, 100)
    if summary:
        chunk_summaries.append(summary)
    time.sleep(1)

Summarizing chunk 1/18...
Summarizing chunk 2/18...
Summarizing chunk 3/18...
Summarizing chunk 4/18...
Summarizing chunk 5/18...
Summarizing chunk 6/18...
Summarizing chunk 7/18...
Summarizing chunk 8/18...
Summarizing chunk 9/18...
Summarizing chunk 10/18...
Summarizing chunk 11/18...
Summarizing chunk 12/18...
Summarizing chunk 13/18...
Summarizing chunk 14/18...
Summarizing chunk 15/18...
Summarizing chunk 16/18...
Summarizing chunk 17/18...
Summarizing chunk 18/18...


In [40]:
combined_summaries = "\n\n".join(chunk_summaries)
final_summary = summarize_text(combined_summaries, 250)

print(final_summary)

Climate change, primarily caused by human activities since the industrial revolution, involves long-term shifts in temperature and weather patterns due to increased greenhouse gases, particularly carbon dioxide. These shifts impact Earth's energy balance, causing effects like polar ice melting, sea level rises, and extreme weather events. Coastal communities are at risk of flooding and erosion, while agriculture faces challenges from changing weather patterns, affecting crop yields and food security. Climate change intensifies heatwaves, hurricanes, and wildfires, threatening ecosystems and biodiversity. Ocean acidification endangers marine life, especially coral reefs and shellfish.

The international community has recognized the urgency of this issue, leading to agreements like the Paris Agreement, which aims to limit warming. Mitigation strategies involve reducing greenhouse gas emissions, while adaptation strategies, like managing risks and improving resilience, are essential for m